# **Chatbot using Seq2Seq LSTM models**

This project is to create conversational chatbot using Sequence to sequence LSTM models. 
Sequence to sequence learning is about training models to convert from one domain to sequences another domain.

# Step 1: Import all the packages 

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
import pandas as pd

## Reading the data

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
datapath = '/content/drive/MyDrive/Chatbot/clean_conversation.txt'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
input_texts = []
target_texts = []
with open(datapath, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(600, len(lines) - 1)]:
    input_text = line.split('\t')[0]
    target_text = line.split('\t')[1]
    input_texts.append(input_text)
    target_texts.append(target_text)

In [4]:
len(input_texts)

572

In [5]:
len(target_texts)

572

In [6]:
zippedList =  list(zip(input_texts, target_texts))
lines = pd.DataFrame(zippedList, columns = ['input' , 'output']) 

In [7]:
lines.head()

,input,output
0,What are your interests,I am interested in all kinds of things. We can...
1,What are your favorite subjects,"My favorite subjects include robotics, compute..."
2,What are your interests,"I am interested in a wide variety of topics, a..."
3,What is your number,I don't have any number
4,What is your number,23 skiddoo!


# Step 3: Preprocessing the data

In [8]:
input_lines = list()
for line in lines.input:
    input_lines.append( line ) 

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( input_lines ) 
tokenized_input_lines = tokenizer.texts_to_sequences( input_lines ) 

length_list = list()
for token_seq in tokenized_input_lines:
    length_list.append( len( token_seq ))
max_input_length = np.array( length_list ).max()
print( 'Input max length is {}'.format( max_input_length ))

padded_input_lines = preprocessing.sequence.pad_sequences( tokenized_input_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_input_lines )
print( 'Encoder input data shape -> {}'.format( encoder_input_data.shape ))

input_word_dict = tokenizer.word_index
num_input_tokens = len( input_word_dict )+1
print( 'Number of Input tokens = {}'.format( num_input_tokens))
np.save('input_word_dict',input_word_dict)

Input max length is 22
Encoder input data shape -> (572, 22)
Number of Input tokens = 552


## Preparing input data for the Decoder

In [9]:
output_lines = list()
for line in lines.output:
    output_lines.append( '<START> ' + line + ' <END>' )  

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( output_lines ) 
tokenized_output_lines = tokenizer.texts_to_sequences( output_lines ) 

length_list = list()
for token_seq in tokenized_output_lines:
    length_list.append( len( token_seq ))
max_output_length = np.array( length_list ).max()
print( 'Output max length is {}'.format( max_output_length ))

padded_output_lines = preprocessing.sequence.pad_sequences( tokenized_output_lines , maxlen=max_output_length, padding='post' )
decoder_input_data = np.array( padded_output_lines )
print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))

output_word_dict = tokenizer.word_index
num_output_tokens = len( output_word_dict )+1
print( 'Number of Output tokens = {}'.format( num_output_tokens))
np.save('output_word_dict',output_word_dict)

Output max length is 74
Decoder input data shape -> (572, 74)
Number of Output tokens = 1717


## Preparing target data for the Decoder 

In [10]:
decoder_target_data = list()
for token_seq in tokenized_output_lines:
    decoder_target_data.append( token_seq[ 1 : ] ) 
    
padded_output_lines = preprocessing.sequence.pad_sequences( decoder_target_data , maxlen=max_output_length, padding='post' )
onehot_output_lines = utils.to_categorical( padded_output_lines , num_output_tokens )
decoder_target_data = np.array( onehot_output_lines )
print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))

Decoder target data shape -> (572, 74, 1717)


# Step 4: Defining Encoder Decoder Model

In [11]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_input_tokens, 256 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 256 , return_state=True , recurrent_dropout=0.2 , dropout=0.2 )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_output_tokens, 256 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 256 , return_state=True , return_sequences=True , recurrent_dropout=0.2 , dropout=0.2)
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_output_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    141312      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 256)    439552      ['input_2[0][0]']                
                                                                                              

# Step 5: Training the Model

We train the model for a number of epochs with RMSprop optimizer and categorical_crossentropy loss function.

In [12]:
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=124, epochs=350) 
model.save( 'model.h5' ) 

Epoch 1/350
5/5 [==============================] - 10s 502ms/step - loss: 1.4536 - accuracy: 0.0726
Epoch 2/350
5/5 [==============================] - 3s 513ms/step - loss: 1.4449 - accuracy: 0.1464
Epoch 3/350
5/5 [==============================] - 2s 493ms/step - loss: 1.3651 - accuracy: 0.0992
Epoch 4/350
5/5 [==============================] - 3s 508ms/step - loss: 1.2025 - accuracy: 0.0702
Epoch 5/350
5/5 [==============================] - 3s 508ms/step - loss: 1.1511 - accuracy: 0.0696
Epoch 6/350
5/5 [==============================] - 2s 489ms/step - loss: 1.1328 - accuracy: 0.0696
Epoch 7/350
5/5 [==============================] - 3s 494ms/step - loss: 1.1190 - accuracy: 0.0783
Epoch 8/350
5/5 [==============================] - 2s 487ms/step - loss: 1.1133 - accuracy: 0.0940
Epoch 9/350
5/5 [==============================] - 2s 491ms/step - loss: 1.1066 - accuracy: 0.1035
Epoch 10/350
5/5 [==============================] - 2s 488ms/step - loss: 1.1007 - accuracy: 0.1085
Epoch 11

# Step 6: Defining Inference Models

Encoder Inference Model: Takes questions as input and outputs LSTM states (h and c)

Decoder Inference Model: Takes in 2 inputs one are the LSTM states, second are the answer input sequences. it will o/p the answers for questions which fed to the encoder model and it's state values.

In [13]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=(256,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(256,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


# Step 7: Talking with the Chatbot

define a method str_to_tokens which converts str questions to Integer tokens with padding.

1. First, we take a question as input and predict the state values using enc_model.
2. We set the state values in the decoder's LSTM.
3. Then, we generate a sequence which contains the <start> element.
4. We input this sequence in the dec_model.
5. We replace the <start> element with the element which was predicted by the dec_model and update the state values.
6. We carry out the above steps iteratively till we hit the <end> tag or the maximum answer length.


In [14]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( input_word_dict[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')


In [16]:
enc_model , dec_model = make_inference_models()

enc_model.save( 'enc_model.h5' ) 
dec_model.save( 'dec_model.h5' ) 
model.save( 'model.h5' ) 

for epoch in range( encoder_input_data.shape[0] ):
    states_values = enc_model.predict( str_to_tokens( input( 'User: ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = output_word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in output_word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( "Bot:" +decoded_translation.replace(' end', '') )
    print()
    


Bot: hello



KeyboardInterrupt: ignored

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>